In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from pathlib import Path
from dask.distributed import Client
import dask.dataframe as dd
from src.data_cleaning import clean_df, lemmatize

In [3]:
RAW_FILE = "sample50k_topic4.csv"
PROECESSED_PATH = Path("../data/data_processed")
PROCESSED_OUT = PROECESSED_PATH/RAW_FILE.replace('.csv', '')
DF_DASK_PARTITIONS = 20

PROCESSED_OUT.mkdir(exist_ok=True)

In [4]:
client = Client()
client.cluster

LocalCluster(2f931313, 'tcp://127.0.0.1:60262', workers=4, threads=12, memory=15.87 GiB)

In [5]:
df_meta = pd.read_csv(
  Path("../data/subsamples")/RAW_FILE,
  usecols = (0,4),
  nrows=5
  )
df_meta.columns = ["article_id", "article"]
df_meta = df_meta.dropna(subset=['article'])

In [6]:
df_dask = pd.read_csv(
  Path("../data/subsamples")/RAW_FILE,
  usecols = (0,4)
  )
df_dask = dd.from_pandas(df_dask, npartitions=DF_DASK_PARTITIONS)
df_dask.columns = ["article_id", "article"]
df_dask = df_dask.dropna(subset=["article"])
df_dask = df_dask.map_partitions(clean_df, meta=df_meta)
df_dask = df_dask.map_partitions(lemmatize, meta=df_meta)
df_dask.to_json(PROCESSED_OUT/"processed_corpus_chunk_*.json", orient='records', lines=True)

['c:\\Users\\angel\\Documents\\fourthbrain-glg\\data\\data_processed\\sample50k_topic4\\processed_corpus_chunk_00.json',
 'c:\\Users\\angel\\Documents\\fourthbrain-glg\\data\\data_processed\\sample50k_topic4\\processed_corpus_chunk_01.json',
 'c:\\Users\\angel\\Documents\\fourthbrain-glg\\data\\data_processed\\sample50k_topic4\\processed_corpus_chunk_02.json',
 'c:\\Users\\angel\\Documents\\fourthbrain-glg\\data\\data_processed\\sample50k_topic4\\processed_corpus_chunk_03.json',
 'c:\\Users\\angel\\Documents\\fourthbrain-glg\\data\\data_processed\\sample50k_topic4\\processed_corpus_chunk_04.json',
 'c:\\Users\\angel\\Documents\\fourthbrain-glg\\data\\data_processed\\sample50k_topic4\\processed_corpus_chunk_05.json',
 'c:\\Users\\angel\\Documents\\fourthbrain-glg\\data\\data_processed\\sample50k_topic4\\processed_corpus_chunk_06.json',
 'c:\\Users\\angel\\Documents\\fourthbrain-glg\\data\\data_processed\\sample50k_topic4\\processed_corpus_chunk_07.json',
 'c:\\Users\\angel\\Documents\\f

In [7]:
client.close()